In [1036]:
import pandas as pd
import json
import re
from functools import reduce
from datetime import datetime

## Data

In [1038]:
pd.set_option('display.max_colwidth', None)

In [1039]:
vacancies = pd.read_csv('vacancies_to_hackaton.csv', delimiter=';')
vacancies = vacancies.drop_duplicates(subset=['id'])

In [1040]:
display(vacancies)

,id,name,area.name,city,company.id,company,company_link,publication_date,salary_from,salary_currency,employment.name,schedule.name,experience.name,key_skills,specializations,description
0,17016760,Официант,Санкт-Петербург,Санкт-Петербург,1905558,2Т,https://hh.ru/employer/1905558,2016-06-16T11:22:58+0300,30000.0,RUR,Полная занятость,Полный день,Нет опыта,NaN,22.193 15.313,"Мы ищем активного, жизнерадостного и дружелюбного официанта. Нашей небольшой, но очень дружной и сплоченной команде не хватает 1 человека. Требования: - опыт работы желателен, но это совсем не главное, мы обращаем внимание на личные качества, такие как вежливость и ответственность, смотрим насколько быстро вы обучаетесь и запоминаете новое, есть ли у вас интерес к профессии, желание стараться и все время стремиться к большему. Условия: - мы готовы научить вас высокому сервису, рассказать об основах гастрономии, барной культуре, психологии общения с гостем и работе в коллективе. - график работы с 07-19/ 11-23 (при составлении графика, готовы учитывать ваши пожелания) - заработная плата 2 раза в месяц - различные бонусы и премии Если у вас мало или совсем нет опыта, мы предложим вам программу стажировки, при которой вы сможете работать неполные смены."
1,17759717,Бармен-официант,Санкт-Петербург,unknown,2440865,3Ф Групп,https://hh.ru/employer/2440865,2016-07-15T13:00:14+0300,30000.0,RUR,Полная занятость,Полный день,Нет опыта,Ответственность | Материальная ответственность | Навыки межличностного общения | Работа в команде | Грамотная речь,22.175 22.193,"Обязанности: Приготовление напитков в баре Прием и подача заказов по столикам Работа с кассой Требования: Желание работать и зарабатывать. Опыт работы на подобной должности будет Вашим преимуществом. А вообще всему научим и всему обучим, так что не нужно бояться приходить без опыта работы). Наличие медицинской книжки будет Вашим преимуществом. Условия: График сменный 2\2. Будние дни с 9-00 до 21-00. Выходные дни с 9-00 до 21-00. Оформление по ТК. Оплата: 100 руб/час + % + чаевые"
2,17816582,Администратор ресторана,Санкт-Петербург,Санкт-Петербург,1911500,4ЕС,https://hh.ru/employer/1911500,2016-07-20T17:20:48+0300,45000.0,RUR,Полная занятость,Сменный график,От 3 до 6 лет,NaN,22.329,"Ресторан ""Рандеву"" приглашает на работу администратора зала с опытом работы на этой должности не менее 3-х лет. Обязанности: Контроль работы персонала ресторана Контроль качества предоставляемых услуг и соблюдения сервиса при обслуживании гостей Проведение банкетных мероприятий Обучение и тренинги персонала Создание дружелюбной атмосферы Требования: Обязательны знание стандартов сервиса, организаторские способности, ответственность, коммуникабельность, хорошо поставленная речь. Условия: График работы: вс-чт с 13-до 01-00, пт-сб с 13 до 03-00 Служебное питание."
3,17746005,Кладовщик,Санкт-Петербург,посёлок Шушары,141100,4Ф,https://hh.ru/employer/141100,2016-08-13T09:26:06+0300,38000.0,RUR,Полная занятость,Сменный график,От 1 года до 3 лет,NaN,21.275 21.563,"Обязанности: Прием товара (Кока-Кола) на склад от поставщиков; Распределение принятой продукции по адресам стеллажного хранения до 6 яруса (11,5 метров); Инвентаризация, внесение корректировок в базе данных; Прием возвратной продукции от водителей. Требования: Опыт работы кладовщиком от года; Опыт работы на электроштабеллере (желательно); Опытный пользователь ПК; Ответственность, внимательность. Условия: Оформление по ТК РФ; Своевременные выплаты заработной платы; График работы 1/1 с 20.00 до 20.00 (выходные суббота и воскресенье); Место работы: Московское шоссе 177 А (служебная развозка от м. Купчино, м. Пр.Ветеранов)."
4,17431972,Кладовщик,Санкт-Петербург,поселок Шушары,141100,4Ф,https://hh.ru/employer/141100,2016-07-12T15:22:23+0300,30000.0,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563 21.564,Обязанности: отпуск и прием на складе ТМЦ ведение учета в программе 1С сборка наборов для магазинов помощь в погрузо-разгрузочных работах Требования: опыт работы на аналогичн

## Algo

In [1077]:
# Tags by which we split description.
tags = {
    "Обязанности": ["обязан", "задачи"],
    "Требования": ["требуется", "требовани", "пожелан"],
    "Условия": ["условия", "предлагаем", "предложени"]
}

# Tags which we exclude.
bad_tags = {
    "Обязанности": [],
    "Требования": ["требований", "требованиям", "требования безо", "требования по"],
    "Условия": ["условиями"]
}

In [1078]:
def make_list(string):
    up_regex = '[А-Я][а-я][^А-Я]+?[А-Я]{2,}|[А-Я][а-я][^А-Я]+'
    
    dot_list = list(filter(None, string.split('. ')))
    comma_list = list(filter(None, string.split(';')))
    new_list = dot_list if len(dot_list) > len(comma_list) else comma_list
    
    if (len(new_list) <= 2):
        up_new_list = re.findall(up_regex, string)
        new_list = up_new_list if len(up_new_list) > len(new_list) else new_list
        if (len(new_list) <= 1):
            return [string]
        
    for i in range(len(new_list)):
        ex_list = new_list[i].split('!')
        if (len(ex_list) > 1):
            del new_list[i]
            for e in ex_list:
                new_list.insert(i, e)
                i += 1
        i -= len(ex_list)
        
        
    return list(filter(lambda x: x != "" and x != " ", new_list))

In [1115]:
def get_data_from_description(description, tags, bad_tags):
    """Extracting text by key words.
    
    Args:
        description (string): text we want to extract
                              tags from.
        tags (dict): keys are tags we want to extract;
                     values are lists of words that
                         match this tag.
    
    Returns:
        data_from_description (dict): 
                     keys are tags;
                     values are text that match key.
    """
    
    # Merging all keywords from tags to 1 list.
    keywords = reduce(lambda x, y: x + y, list(tags.values()))
    bad_keywords = reduce(lambda x, y: x + y, list(bad_tags.values()))
    
    # Splitting description by keywords. 
    part = []
    all_parts = []
    for word in description.split():
        if any(subword in word.lower() for subword in keywords) and (not (any(bad_subword in word.lower() for bad_subword in bad_keywords))):
            all_parts.append(part)
            all_parts.append([word])
            part = []
        else:
            part.append(word)
    all_parts.append(part)

    # Merging lists of strings to strings.
    all_strings = []
    for word_list in all_parts:
        all_strings.append(' '.join(word_list))

    # Creating dictionary with keys same as tags keys and
    # values as all strings matching these keys
    # (value is a text that follows current key)
    data_from_description = {}
    for key in tags.keys():
        data_from_description[key] = []
    
    for string in all_strings:
        for key in tags.keys():
            if any(subword in string.lower() for subword in tags[key]) and (not (any(bad_subword in string.lower() for bad_subword in bad_keywords))):
                data_from_description[key] = make_list(all_strings[all_strings.index(string) + 1])

    return all_strings, data_from_description

In [1130]:
def get_json_from_data( id ):
    record = vacancies[vacancies["id"] == int(id)]
    record = record.drop(['id','company.id', 'company_link', 'salary_currency', 'key_skills', 'specializations', 'description'], axis=1)
    dict_record = record.to_dict('records')[0]
    
    dict_record['q0']=dict_record.pop('name')
    dict_record['q1']=dict_record.pop('area.name')
    dict_record['q2']=dict_record.pop('city')
    if pd.isnull(dict_record['q2']) or (dict_record['q2'] == 'unknown'):
        dict_record['q2'] = dict_record['q1']
    dict_record['q3']=dict_record.pop('company')
    dict_record['q4']=int(dict_record.pop('salary_from'))
    dict_record['q5']=dict_record.pop('employment.name')
    dict_record['q6']=dict_record.pop('schedule.name')
    dict_record['q7']=dict_record.pop('experience.name')
    dict_record['q8']=dict_record.pop('publication_date')
    match = re.search(r'\d{4}-\d{2}-\d{2}', dict_record['q8'])
    dict_record['q8'] = abs((datetime.strptime("2020-09-20", '%Y-%m-%d').date() - datetime.strptime(match.group(), '%Y-%m-%d').date()).days)
    
    description = list(vacancies[vacancies["id"] == int(id)]["description"])[0]
    all_strings, description_data = get_data_from_description(description, tags, bad_tags)
    dict_record['q9'] = description_data["Требования"]
    dict_record['q10'] = description_data["Обязанности"]
    dict_record['q11'] = description_data["Условия"]
    
    predict_json = {'common': dict_record}
    dict_json = {f'{id}': predict_json}
    json_record = json.dumps(dict_json,  ensure_ascii=False)
    return json_record

### Questions

In [1134]:
questions_common = {
    "q0" : "Название вакансии",
    "q1" : "Регион",
    "q2" : "Город",
    "q3" : "Название компании",
    "q4" : "Зарплата от",
    "q5" : "Тип занятости",
    "q6" : "Тип графика",
    "q7" : "Опыт",
    "q8" : "Время поиска работы",
    "q9" : "Требования",
    "q10" : "Обязанности",
    "q11" : "Условия"
}

## Testing

### JSON

In [1116]:
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data_json, f, ensure_ascii=False, indent=4)

In [1117]:
with open('data.json', 'r') as f:
    data = json.load(f)

In [1118]:
len(data.keys())

10

In [1119]:
for id in list(data.keys()):
    print("Id:")
    print(id)
    print("\n")
    
    print("Human processed data:")
    print(json.dumps(data[id], indent=4, ensure_ascii=False))
    print("\n")
    
    description = list(vacancies[vacancies["id"] == int(id)]["description"])[0]
    all_strings, data_from_description = get_data_from_description(description, tags, bad_tags)
    
    print("Parsed data:")
    print(json.dumps(data_from_description, indent=4, sort_keys=True, ensure_ascii=False))
    print("\n")
    
    print("Meta data:")
    print(all_strings)
    print("\n")
    
    print("-------------------------------------------------------------------------")

Id:
17735441


Human processed data:
{
    "common": {
        "q0": "Кладовщик",
        "q1": "Санкт-Петербург",
        "q2": "Санкт-Петербург",
        "q3": "Партия еды",
        "q4": "30000",
        "q5": [
            "Полная занятость"
        ],
        "q6": [
            "Полный день"
        ],
        "q7": "1-3",
        "q8": 1460,
        "q9": [
            "знание складских операций и учета владение ПК на уровне уверенного пользователя (пакет офисных программ, 1С)",
            "оперативность в работе",
            "опыт работы с продуктами питания",
            "ответственность",
            "добросовестность",
            "внимательность",
            "аккуратность"
        ],
        "q10": [
            "приемка выдача товаров",
            "ответственность за качество входящих товаров",
            "внесение первичной документации в 1С",
            "организация хранения и учета",
            "отслеживание остатков на складах",
            "участие в проведении

In [1131]:
get_json_from_data("25282392")

'{"25282392": {"common": {"q0": "Кладовщик", "q1": "Санкт-Петербург", "q2": "Санкт-Петербург", "q3": "Гаврилко Вячеслав Владимирович", "q4": 30000, "q5": "Полная занятость", "q6": "Полный день", "q7": "От 1 года до 3 лет", "q8": 898, "q9": ["Ответственное отношение к работе, готовность к активной работе", "Желание работать и зарабатывать Пунктуальность, аккуратность, честность, обучаемость, хорошие адаптивные качества, готовность следовать установленным правилам и процедурам", "Умение выполнять несколько дел одновременно, высокая концентрация и уровень распределения внимания", "Способность быстро принимать решения и реагировать на информацию."], "q10": ["Приемка товара на складе", "Размещение товара на места хранения", "Сборка заказов с мест хранения", "Упаковка заказов для отгрузки."], "q11": ["заработная плата состоит из оклада и переменной части обучение в процессе работы"]}}}'